In [1]:
# Step 1: Install required packages
!pip install -q bitsandbytes>=0.41.0
!pip install -q datasets accelerate loralib transformers peft

In [2]:
# Step 2: Import necessary libraries
import os
import torch
import json
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from transformers import Trainer, DataCollatorForLanguageModeling
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Step 3: Set up GPU and environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use first GPU
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Avoid warnings

In [5]:
# Step 4: Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,  # Double quantization for more memory efficiency
    bnb_4bit_quant_type="nf4",       # Normal float 4-bit quantization for better accuracy
    bnb_4bit_compute_dtype=torch.float16  # Compute in fp16 for better performance
)

In [ ]:
# Step 5: Loading the Lughaat model with 4-bit quantization
print("Loading base model...")
model = AutoModelForCausalLM.from_pretrained(
    "muhammadnoman76/Lughaat-1.0-8B-Instruct",
    quantization_config=bnb_config,
    device_map="auto",    #automatically determine device mapping
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("muhammadnoman76/Lughaat-1.0-8B-Instruct")

Loading base model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/947 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [7]:
# Ensure the tokenizer has padding token, EOS token and set left padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

In [8]:
# Step 6: Prepare model for QLoRA training
print("Preparing model for QLoRA fine-tuning...")
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

Preparing model for QLoRA fine-tuning...


In [9]:
# Step 7: Define LoRA configuration
lora_config = LoraConfig(
    r=16,                   # Rank dimension
    lora_alpha=32,          # Alpha parameter for LoRA scaling
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention modules
        "gate_proj", "up_proj", "down_proj"      # MLP modules
    ],
    lora_dropout=0.05,      # Dropout probability for LoRA layers
    bias="none",            # Don't train bias parameters
    task_type="CAUSAL_LM"   # Task type - causal language modeling
)

In [10]:
# Step 8: Apply LoRA to model
print("Applying LoRA adapters...")
model = get_peft_model(model, lora_config)

Applying LoRA adapters...


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [11]:
# Print trainable parameters percentage
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} || "
        f"All params: {all_param} || "
        f"Trainable%: {100 * trainable_params / all_param:.2f}%"
    )

print_trainable_parameters(model)

Trainable params: 41943040 || All params: 5633216512 || Trainable%: 0.74%


In [12]:
# Step 9: Load and prepare the Urdu QA dataset
from datasets import DatasetDict, Dataset

print("Loading dataset...")
dataset_path = "/content/drive/MyDrive/train_data/train_qa_data_1.json"

class Llama3InstructDataset:
    def __init__(self, data):
        self.data = data
        self.prompts = []
        self.create_prompts()

    def create_prompt(self, row):
        instruction = "آپ ایک اردو میں سوال و جواب کرنے والا مددگار ہیں۔ دیے گئے علم کی بنیاد پر سوال کا درست جواب دیں"

        # Format with Llama 3 tokens
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>{instruction}<|eot_id|><|start_header_id|>user<|end_header_id|>علم: {row['knowledge']}\n\nسوال: {row['question']}<|eot_id|><|start_header_id|>assistant<|end_header_id|>{row['right_answer']}<|eot_id|>"""
        return prompt

    def create_prompts(self):
        for row in self.data:
            prompt = self.create_prompt(row)
            self.prompts.append(prompt)

    def get_dataset(self):
        df = pd.DataFrame({'text': self.prompts})
        return df

# Load dataset
with open(dataset_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Create formatted dataset
dataset_formatter = Llama3InstructDataset(data)
df = dataset_formatter.get_dataset()
dataset = Dataset.from_pandas(df)

print(f"Dataset size: {len(dataset)} examples")
print("Sample entry:")
print(dataset[0]['text'][:500] + "...")  # Print beginning of first example

Loading dataset...
Dataset size: 4000 examples
Sample entry:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>آپ ایک اردو میں سوال و جواب کرنے والا مددگار ہیں۔ دیے گئے علم کی بنیاد پر سوال کا درست جواب دیں<|eot_id|><|start_header_id|>user<|end_header_id|>علم: سیزن کے آغاز کے قریب ، ٹمبر وولفز این بی اے کی تاریخ میں پہلی ٹیم ہوگی جس میں اینڈریو وگنس ، زاک لا وائن ، کارل اینتھونی ٹاؤنز اور ٹائس جونز کے درمیان چار کھلاڑی جو 20 یا اس سے کم عمر کے تھے ، ایک سیزن شروع کرنے کے لئے شامل ہوں گے۔ انہوں نے کنٹکی یونیورسٹی کے لئے کالج باسکٹ بال کھیلا۔

سوال...


In [13]:
class CustomDataCollator:
    def __init__(self, tokenizer, max_length=512):  # Add max_length parameter with a default value
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, examples):
        # Process the text field from examples
        inputs = self.tokenizer([example["text"] for example in examples],
                              padding=True,
                              truncation=True,
                              max_length=self.max_length,
                              return_tensors="pt")

        # Set up labels for causal language modeling
        inputs["labels"] = inputs["input_ids"].clone()

        return inputs

In [14]:
# Step 11: Initialize trainer with your custom collator
training_args = TrainingArguments(
    output_dir="./results/urdu-qa-model_2",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_dir="./logs",
    logging_steps=1,
    save_strategy="epoch",
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    seed=42,
    push_to_hub=False,
    remove_unused_columns=False,
    optim="adamw_bnb_8bit"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=CustomDataCollator(tokenizer, max_length=512)
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [15]:
# Step 13: Disable cache for training (to save memory)
model.config.use_cache = False

In [16]:
# Step 14: Train the model
print("Starting training...")
trainer.train()

Starting training...


Step,Training Loss
1,2.209700
2,2.318000
3,2.268400
4,2.097600
5,2.052800
6,1.815100
7,1.704700
8,1.640200
9,1.622900
10,1.445200


TrainOutput(global_step=250, training_loss=1.1523262984752656, metrics={'train_runtime': 9305.3765, 'train_samples_per_second': 0.43, 'train_steps_per_second': 0.027, 'total_flos': 6.620255288276582e+16, 'train_loss': 1.1523262984752656, 'epoch': 1.0})

In [ ]:
# Step 15: Save the fine-tuned model
output_dir = "/content/drive/MyDrive/fine-tuning/fine_tuned_model-2"

#createthe directory if it doesn't exist
import os
os.makedirs(output_dir, exist_ok=True)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved to {output_dir}")

Model saved to /content/drive/MyDrive/fine-tuning/fine_tuned_model-2


In [19]:
# Step 16: Optional - Push to Hugging Face Hub

from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Push adapter weights and tokenizer to HF Hub
model.push_to_hub("zenu12/lughaat-finetuned-qa")
tokenizer.push_to_hub("zenu12/lughaat-finetuned-qa")

adapter_model.safetensors:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/umerbappi/alif-finetuned-qa/commit/692344ef75e1d7b212fbbdc28f7af1d1133fdfda', commit_message='Upload tokenizer', commit_description='', oid='692344ef75e1d7b212fbbdc28f7af1d1133fdfda', pr_url=None, repo_url=RepoUrl('https://huggingface.co/umerbappi/alif-finetuned-qa', endpoint='https://huggingface.co', repo_type='model', repo_id='umerbappi/alif-finetuned-qa'), pr_revision=None, pr_num=None)